This method works in lieu of just instantiating the ReAct agent. But they do the same thing.

In [7]:
import json
import os
from typing import Sequence, List

from llama_index.llms.bedrock import Bedrock
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [5]:
from llama_index.core.agent import AgentRunner
from llama_index.core.agent.react import ReActAgentWorker

In [8]:
llm = Bedrock(
    model = "anthropic.claude-3-opus-20240229-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)

In [9]:
react_step_engine = ReActAgentWorker.from_tools(tools, llm=llm, verbose=True)
agent = AgentRunner(react_step_engine)

In [10]:
agent.chat("Hi")

Thought: The current language of the user is: English. I don't need to use a tool to respond to this greeting.
Answer: Hello! How can I assist you today?


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[], is_dummy_stream=False)

In [11]:
response = agent.chat("What is (121 * 3) + 42?")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Thought: I now have the result of 121 * 3, which is 363. I still need to add 42 to this result to get the final answer.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of (121 * 3) + 42 is 405.


In [12]:
tasks = agent.list_tasks()
print(len(tasks))

2
